In [1]:
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from tensorflow.keras.optimizers import RMSprop
import os
import random
import shutil

In [2]:
base_dir = "./dataset"
sub_directories = os.listdir(base_dir)
print(len(sub_directories))
print(sub_directories)

19
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'dec', 'div', 'eq', 'mul', 'sub', 'x', 'y', 'z']


In [3]:
# Creating folders for train/ test set

inp = input("Are you sure you wanna make train and test folders again? ")

if inp == "n":

    pass

else :

    base_dir = "./dataset"
    sub_directories = os.listdir(base_dir)

    train_dir = "train_dataset"
    test_dir = "test_dataset"

    os.mkdir(train_dir)
    os.mkdir(test_dir)

    for i in sub_directories:

        os.mkdir(os.path.join(train_dir, i))
        os.mkdir(os.path.join(test_dir, i))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'train_dataset'

In [ ]:
# Code to split the entire dataset into train and test sets

input("Are you sure you wanna split the entire dataset again? ")

train_ratio = 0.9
test_ratio = 0.1

for i in sub_directories:

    path = os.path.join(base_dir, i)

    sub_dir_files = os.listdir(path)
    len_sub_dir = len(sub_dir_files)
    train_sub_dir_count = int(len_sub_dir*train_ratio)

    train_files = random.sample(sub_dir_files, train_sub_dir_count)

    test_files = [i for i in sub_dir_files if i not in train_files]

    for j in train_files:

        shutil.copy(os.path.join(base_dir, i, j), os.path.join(train_dir, i, j))

    for j in test_files:

        shutil.copy(os.path.join(base_dir, i, j), os.path.join(test_dir, i, j))

In [19]:
# Generating the model

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    "./test_dataset/",
    target_size = (100, 100),
    batch_size = 128,
    class_mode = "categorical"
)
test_generator = test_datagen.flow_from_directory(
    "./test_dataset/",
    target_size = (100, 100),
    batch_size = 128,
    class_mode = "categorical"
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = "relu", input_shape = (100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    
    tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(19, activation = "softmax")
])

model.compile(
    optimizer = RMSprop(learning_rate = 0.001),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

history = model.fit(
    train_generator,
    epochs = 20,
    validation_data = test_generator,
    verbose = 2
)


Found 1018 images belonging to 19 classes.
Found 1018 images belonging to 19 classes.
Epoch 1/20
8/8 - 7s - loss: 0.3125 - accuracy: 0.0668 - val_loss: 0.2025 - val_accuracy: 0.1238 - 7s/epoch - 859ms/step
Epoch 2/20
8/8 - 6s - loss: 0.2014 - accuracy: 0.2024 - val_loss: 0.1794 - val_accuracy: 0.3988 - 6s/epoch - 698ms/step
Epoch 3/20
8/8 - 6s - loss: 0.1740 - accuracy: 0.4047 - val_loss: 0.1553 - val_accuracy: 0.4804 - 6s/epoch - 690ms/step
Epoch 4/20
8/8 - 5s - loss: 0.1482 - accuracy: 0.4578 - val_loss: 0.1334 - val_accuracy: 0.5373 - 5s/epoch - 667ms/step
Epoch 5/20
8/8 - 5s - loss: 0.1298 - accuracy: 0.5540 - val_loss: 0.1212 - val_accuracy: 0.6297 - 5s/epoch - 673ms/step
Epoch 6/20
8/8 - 5s - loss: 0.1196 - accuracy: 0.6287 - val_loss: 0.0982 - val_accuracy: 0.6601 - 5s/epoch - 679ms/step
Epoch 7/20
8/8 - 5s - loss: 0.1004 - accuracy: 0.6670 - val_loss: 0.0885 - val_accuracy: 0.7515 - 5s/epoch - 687ms/step
Epoch 8/20
8/8 - 6s - loss: 0.0915 - accuracy: 0.7191 - val_loss: 0.0665 -

In [20]:
# For testing this model personally, open Microsoft paint, make the canvas size 100 * 100 (for better prediction, and draw math symbols)

img = load_img("./self_checking/9.png", target_size = (100, 100))
x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Scale by 1/255
x /= 255.0

# Run the image through the network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = model.predict(x)

1/1 [==============================] - 0s 99ms/step


In [ ]:
class_details = {}
counter = 0
for i in sub_directories:

    class_details[counter] = i
    counter += 1

class_details

In [21]:
class_details[np.where(successive_feature_maps[0] == max(successive_feature_maps[0]))[0][0]]

# If the model works shit, please forgive me 😁

'3'